In [1]:
import sys
import os
%load_ext autoreload
%autoreload 2

sys.path.append('../')
os.chdir('../')

In [2]:
import re

string = "res=[0, 1, 2, 5]"
pattern = r"res=\[(.*?)\]"
result = re.findall(pattern, string)[0]
lst = [int(num) for num in result.split(", ")]
print(lst)


[0, 1, 2, 5]


In [12]:
from ast import literal_eval
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import re



global_path = './outputs'
# all_result_paths = glob.glob(global_path + '/parity*_exp/**/result.csv', recursive=True)
all_result_paths = glob.glob(global_path + '/mod3*_exp/**/result.csv', recursive=True)
result_summary = {}
n_layers = 6
for result_path in all_result_paths:
    result = pd.read_csv(result_path)
    top_test_result = result['test_acc'].max()
    pe_type = 'NoPE' if 'nope' in result_path else 'Original'
    pattern = r"res=\[(.*?)\]"
    residual_list = re.findall(pattern, result_path)[0]
    residual_list = [int(num) for num in residual_list.split(", ")] if residual_list != '' else []
    # residual_list = literal_eval(residual_list)
    missing_rc = [i for i in range(n_layers) if i not in residual_list]
    missing_rc_str = '{' + ','.join([str(i) for i in missing_rc])+ '}'


    if pe_type not in result_summary:
        result_summary[pe_type] = {}
    if missing_rc_str not in result_summary[pe_type]:
        result_summary[pe_type][missing_rc_str] = []
    result_summary[pe_type][missing_rc_str].append(top_test_result)
    if pe_type == 'Original' and missing_rc_str == '{5}':
        print(result_path, pe_type, missing_rc_str, top_test_result)


In [13]:
from ast import literal_eval
mean_results = {}
for pe_type in result_summary:
    cur_type_dict = result_summary[pe_type].copy()
    for missing_rc in cur_type_dict:
        cur_type_dict[missing_rc] = len(cur_type_dict[missing_rc])
    mean_results[pe_type] = cur_type_dict
df = pd.DataFrame(mean_results)
missing0 = ['{}']
missing1 = [str({i}) for i in range(n_layers)]
missing2 = ['{' + ','.join([str(i), str(i+1)])+ '}'for i in range(n_layers-1)]
missing3 = ['{' + ','.join([str(i), str(i+1), str(i+2)])+ '}'for i in range(n_layers-2)]
from IPython.display import display
display(df.loc[missing0,:].T)
display(df.loc[missing1,:].T)
display(df.loc[missing2,:].T)
display(df.loc[missing3,:].T)

,{}
NoPE,1.0
Original,NaN


KeyError: "None of [Index(['{0}', '{1}', '{2}', '{3}', '{4}', '{5}'], dtype='object')] are in the [index]"

In [5]:
from ast import literal_eval
from collections import OrderedDict
final_results = OrderedDict()
for pe_type in {'Original', 'NoPE'}:
    cur_type_dict = result_summary[pe_type].copy()
    final_dict = {
        'min': cur_type_dict.copy(),
        'max': cur_type_dict.copy(),
        'avg.': cur_type_dict.copy(),
    }
    for missing_rc in cur_type_dict:
        final_dict['avg.'][missing_rc] = np.mean(cur_type_dict[missing_rc])
        final_dict['min'][missing_rc] = np.min(cur_type_dict[missing_rc])
        final_dict['max'][missing_rc] = np.max(cur_type_dict[missing_rc])

    for k in final_dict:
        final_results[pe_type+' ('+k+')'] = final_dict[k]
df = pd.DataFrame(final_results)
missing1 = [str({i}) for i in range(n_layers)]
missing2 = ['{' + ','.join([str(i), str(i+1)])+ '}'for i in range(n_layers-1)]
missing3 = ['{' + ','.join([str(i), str(i+1), str(i+2)])+ '}'for i in range(n_layers-2)]
missing4 = ['{' + ','.join([str(i), str(i+1), str(i+2), str(i+3)])+ '}'for i in range(n_layers-3)]
missing5 = ['{' + ','.join([str(i), str(i+1), str(i+2), str(i+3), str(i+4)])+ '}'for i in range(n_layers-4)]
from IPython.display import display
r1 = df.loc[missing1,:].T
r2 = df.loc[missing2,:].T
r3 = df.loc[missing3,:].T
r4 = df.loc[missing4,:].T
r5 = df.loc[missing5,:].T
r0 = df.loc[['{}'],:].T
display(r0)
display(r1)
display(r2)
display(r3)
display(r4)
display(r5)

# write all 3 to the same csv file
all_r = pd.concat([r0, r1, r2, r3], axis=1)
all_r.to_csv('all_r_mean.csv')


,{}
NoPE (min),97.60
NoPE (max),99.65
NoPE (avg.),99.32
Original (min),98.50
Original (max),99.95
Original (avg.),99.56


,{0},{1},{2},{3},{4},{5}
NoPE (min),96.800,91.950,99.650,94.85,99.650,99.700
NoPE (max),99.650,99.650,99.950,99.95,100.000,99.950
NoPE (avg.),98.570,97.550,99.845,99.44,99.925,99.865
Original (min),96.700,99.600,99.450,99.60,99.650,98.900
Original (max),99.650,99.950,99.950,99.95,99.950,99.950
Original (avg.),99.025,99.715,99.670,99.86,99.915,99.665


,"{0,1}","{1,2}","{2,3}","{3,4}","{4,5}"
NoPE (min),99.70,99.650,99.650,98.100,50.05
NoPE (max),99.95,99.950,99.950,100.000,100.00
NoPE (avg.),99.91,99.905,99.865,99.555,92.39
Original (min),99.95,93.800,99.700,98.800,94.25
Original (max),99.95,100.000,99.950,99.950,99.95
Original (avg.),99.95,99.175,99.900,99.730,98.98


,"{0,1,2}","{1,2,3}","{2,3,4}","{3,4,5}"
NoPE (min),99.65,74.750,68.70,49.300
NoPE (max),99.95,99.950,99.95,99.600
NoPE (avg.),99.86,96.115,95.50,59.575
Original (min),99.60,97.650,69.00,49.300
Original (max),99.95,99.950,99.95,69.400
Original (avg.),99.89,99.530,96.25,51.955


,"{0,1,2,3}","{1,2,3,4}","{2,3,4,5}"
NoPE (min),95.00,49.30,49.300
NoPE (max),99.95,89.05,49.750
NoPE (avg.),99.08,54.77,49.345
Original (min),95.90,49.30,49.300
Original (max),99.95,53.55,51.900
Original (avg.),98.57,50.39,49.955


,"{0,1,2,3,4}","{1,2,3,4,5}"
NoPE (min),49.300,49.300
NoPE (max),51.200,50.750
NoPE (avg.),50.090,49.465
Original (min),49.300,49.300
Original (max),51.750,50.650
Original (avg.),50.105,49.610
